Importing neccesary libraries and objects
==================================

In [23]:
import math
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta

Reading and prepairing data
=======================

In [24]:
df = pd.read_csv('swissmetro.dat', sep='\t')
df

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE
0,2,0,1,1,1,0,1,1,0,3,...,112,48,120,63,52,20,0,117,65,2
1,2,0,1,1,1,0,1,1,0,3,...,103,48,30,60,49,10,0,117,84,2
2,2,0,1,1,1,0,1,1,0,3,...,130,48,60,67,58,30,0,117,52,2
3,2,0,1,1,1,0,1,1,0,3,...,103,40,30,63,52,20,0,72,52,2
4,2,0,1,1,1,0,1,1,0,3,...,130,36,60,63,42,20,0,90,84,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,3,1,1,1192,4,1,7,1,0,5,...,148,13,30,93,17,30,0,156,56,2
10724,3,1,1,1192,4,1,7,1,0,5,...,148,12,30,96,16,10,0,96,70,3
10725,3,1,1,1192,4,1,7,1,0,5,...,148,16,60,93,16,20,0,96,56,3
10726,3,1,1,1192,4,1,7,1,0,5,...,178,16,30,96,17,30,0,96,91,2


In [25]:
database = db.Database("swissmetro ",df)
globals().update(database.variables)

exclude = ((PURPOSE != 1) *
           (PURPOSE != 3) +
           (CHOICE == 0)) > 0
database.remove(exclude)

daf = database.data
daf

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE
0,2,0,1,1,1,0,1,1,0,3,...,112,48,120,63,52,20,0,117,65,2
1,2,0,1,1,1,0,1,1,0,3,...,103,48,30,60,49,10,0,117,84,2
2,2,0,1,1,1,0,1,1,0,3,...,130,48,60,67,58,30,0,117,52,2
3,2,0,1,1,1,0,1,1,0,3,...,103,40,30,63,52,20,0,72,52,2
4,2,0,1,1,1,0,1,1,0,3,...,130,36,60,63,42,20,0,90,84,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8446,3,1,1,939,3,1,7,3,1,5,...,108,13,30,50,17,30,0,130,64,1
8447,3,1,1,939,3,1,7,3,1,5,...,108,12,30,53,16,10,0,80,80,1
8448,3,1,1,939,3,1,7,3,1,5,...,108,16,60,50,16,20,0,80,64,1
8449,3,1,1,939,3,1,7,3,1,5,...,128,16,30,53,17,30,0,80,104,1


Specifiing model
==============

In [54]:
# Defining Betas
ASC_CAR = Beta('ASC_CAR', 0, None, None, 1)
ASC_TRAIN = Beta('ASC_TRAIN', 0, None, None, 0)
ASC_SM = Beta('ASC_SM', 0, None, None, 0)
B_TIME = Beta('B_TIME', 0, None, None, 0)
B_COST = Beta('B_COST', 0, None, None, 0)

# Defining new variables
scale = 100
SM_COST = SM_CO * (GA == 0)
TRAIN_COST = TRAIN_CO * (GA == 0)
CAR_AV_SP = CAR_AV * (SP != 0)
TRAIN_AV_SP = TRAIN_AV * (SP != 0)
TRAIN_TT_SCALED = TRAIN_TT / scale
TRAIN_COST_SCALED = TRAIN_COST / scale
SM_TT_SCALED = SM_TT / scale
SM_COST_SCALED = SM_COST / scale
CAR_TT_SCALED = CAR_TT / scale
CAR_CO_SCALED = CAR_CO / scale

# Utility functions
V1 = ASC_TRAIN + B_TIME * TRAIN_TT_SCALED + B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + B_TIME * SM_TT_SCALED + B_COST * SM_COST_SCALED
V3 = ASC_CAR + B_TIME * CAR_TT_SCALED + B_COST * CAR_CO_SCALED

# Associating numbers to utility funtions
V = {1: V1,
     2: V2,
     3: V3}

# Associating availability
av = {1: TRAIN_AV_SP,
      2: SM_AV,
      3: CAR_AV_SP}

# Log probability
logprob = models.loglogit(V, av, CHOICE)

In [55]:
the_biogeme = bio.BIOGEME(database, logprob, skipAudit=True)
the_biogeme.modelName = 'logit'

Estimation
=========

In [56]:
results = the_biogeme.estimate()
pandasResults = results.getEstimatedParameters()
pandasResults

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_SM,0.154637,0.058163,2.658665,0.007845
ASC_TRAIN,-0.546533,0.048957,-11.163537,0.000000
B_COST,-1.083788,0.068225,-15.885519,0.000000
B_TIME,-1.277850,0.104254,-12.257116,0.000000


Computing probability of choosing each mode of transport
================================================

### If person has GA card, then riding by train or sm is free. Code below calulates mean cost of each service using that information.


In [57]:
tdf = daf.filter(items=['TRAIN_CO', 'GA'])
smdf = daf.filter(items=['SM_CO', 'GA'])
tdf

,TRAIN_CO,GA
0,48,0
1,48,0
2,48,0
3,40,0
4,36,0
...,...,...
8446,13,0
8447,12,0
8448,16,0
8449,16,0


In [58]:
tg = tdf.groupby(['GA']).sum()
tg

,TRAIN_CO
GA,
0,570922
1,2751389


In [59]:
train_cost = tg.at[0, 'TRAIN_CO'] / len(tdf) / scale
train_cost

0.8435608747044917

In [60]:
sg = smdf.groupby(['GA']).sum()
sg

,SM_CO
GA,
0,692383
1,3646355


In [61]:
sm_cost = sg.at[0, 'SM_CO'] / len(tdf) / scale
sm_cost

1.0230245271867613

### Now, utility of each transportation mode may be estimated

In [62]:
train_utility = B_COST.getValue() * train_cost + B_TIME.getValue() * daf["TRAIN_TT"].mean() / scale + ASC_TRAIN.getValue()
train_utility

-3.5829952170601005

In [63]:
car_utility = B_COST.getValue() * daf["CAR_CO"].mean() / scale + B_TIME.getValue() * daf["CAR_TT"].mean() / scale
car_utility

-2.4261978617547615

In [64]:
sm_utility = B_COST.getValue() * sm_cost + B_TIME.getValue() * daf["SM_TT"].mean() / scale + ASC_SM.getValue()
sm_utility

-2.033983163766515

In [65]:
train_exp = math.exp(train_utility)
car_exp = math.exp(car_utility)
sm_exp = math.exp(sm_utility)

print(train_exp)
print(car_exp)
print(sm_exp)

0.027792329404500122
0.0883721979392379
0.1308134307354787


### Probability when only car and train are availble

In [66]:
p_car = car_exp / (car_exp + train_exp)
p_car

0.76075029064371

In [67]:
p_train = train_exp / (car_exp + train_exp)
p_train

0.23924970935629

### Probability when car, train and SM are availble

In [68]:
p_car = car_exp / (car_exp + train_exp + sm_exp)
p_car

0.3578141087023363

In [69]:
p_train = train_exp / (car_exp + train_exp + sm_exp)
p_train

0.11252959422227428

In [70]:
p_sm = sm_exp / (car_exp + train_exp + sm_exp)
p_sm

0.5296562970753895